## Investigate wflow outputs with xarray

In [ ]:
%matplotlib inline
import xarray as xr
import matplotlib.pyplot as plt

In [ ]:
fn = r'c:\temp\wflow_zambezi\zambezi_4km\test\outmaps.nc'
ds = xr.open_dataset(fn)
ds

In [ ]:
f = plt.figure(figsize=(12,12))
ds_sum = ds['run'][0, :, :].plot()



In [ ]:
lon_watopa = 19.5
lat_watopa = -12.25
ts = ds['run'].sel(lat=lat, lon=lon, method='nearest')
ts.plot()
